In [1]:
!pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/89/1b/12521efcbc6058e2673583bb096c2b5046a9df39bd73eca392c1efed24e5/pandas-2.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for numpy>=1.22.4 from https://files.pythonhosted.org/packages/4b/d7/ecf66c1cd12dc28b4040b15ab4d17b773b87fa9d29ca16125de01adb36cd/numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 7.6 MB/s eta 0:00:00
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/65/58/f9c9e6be752e9fcb8b6a0ee9fb87e6e7a1f6bcab2cdc73f02bb7ba91ada0/tzdata-2024.1-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 6.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 15.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.

In [2]:
!pip install matplotlib

  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/a7/68/16e7b9154fae61fb29f0f3450b39b855b89e6d2c598d67302e70f96883af/matplotlib-3.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/67/0f/6e5b4879594cd1cbb6a2754d9230937be444f404cf07c360c07a10b36aac/contourpy-1.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for cycler>=0.10 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629efb316b96de511b418c53d245aae6/cycler-0.12.1-py3-none-any.whl.metadata
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/e2/f3/7e338ff486517408884d3ab5f12d427d556c8a106a3eb24f48cbb0844efd/fonttools-4.52.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install ipywidgets

In [8]:
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML

def calculate_custom_costs(orders, start_bundles, prepaid_bundles, overage_cost, selected_start, selected_small_prepaids, selected_big_prepaids):
    start_bundle = next(bundle for bundle in start_bundles if bundle[2] == selected_start)
    starter_cost, starter_orders, _ = start_bundle

    if selected_start == 'small':
        selected_prepaids = [(cost, orders) for cost, orders, _ in prepaid_bundles if orders == 250][:selected_small_prepaids]
    else:
        selected_prepaids = [(cost, orders) for cost, orders, _ in prepaid_bundles if orders == 1100][:selected_big_prepaids]
    
    total_bundle_cost = sum(bundle_cost for bundle_cost, bundle_orders in selected_prepaids)
    total_bundle_orders = sum(bundle_orders for bundle_cost, bundle_orders in selected_prepaids)
    
    remaining_orders_after_bundles = max(0, orders - starter_orders - total_bundle_orders)
    overage_cost_total = remaining_orders_after_bundles * overage_cost
    
    total_cost = starter_cost + total_bundle_cost + overage_cost_total
    
    return total_cost, starter_cost, starter_orders, selected_prepaids, remaining_orders_after_bundles, overage_cost_total

def bundle_description(bundle_combo):
    bundle_counts = {}
    for bundle_cost, bundle_orders in bundle_combo:
        if (bundle_cost, bundle_orders) in bundle_counts:
            bundle_counts[(bundle_cost, bundle_orders)] += 1
        else:
            bundle_counts[(bundle_cost, bundle_orders)] = 1
    
    descriptions = []
    for (bundle_cost, bundle_orders), count in bundle_counts.items():
        if count > 1:
            descriptions.append(f"{count}x (€{bundle_cost} for {bundle_orders} orders)")
        else:
            descriptions.append(f"€{bundle_cost} for {bundle_orders} orders")
    
    return ", ".join(descriptions)

def display_custom_costs_df(orders, start_bundles, prepaid_bundles, overage_cost, selected_start, selected_small_prepaids, selected_big_prepaids):
    total_cost, starter_cost, starter_orders, selected_prepaids, remaining_orders_after_bundles, overage_cost_total = calculate_custom_costs(
        orders, start_bundles, prepaid_bundles, overage_cost, selected_start, selected_small_prepaids, selected_big_prepaids
    )
    
    if selected_start == 'small':
        used_starter_bundle = 'small'
    else:
        used_starter_bundle = 'big'
    
    small_starter_description = f"€{starter_cost} for {starter_orders} orders" if used_starter_bundle == 'small' else "Not used"
    big_starter_description = f"€{starter_cost} for {starter_orders} orders" if used_starter_bundle == 'big' else "Not used"
    
    small_prepaid_descriptions = bundle_description([bundle for bundle in selected_prepaids if bundle[1] == 250])
    big_prepaid_descriptions = bundle_description([bundle for bundle in selected_prepaids if bundle[1] == 1100])
    
    overage_cost_str = f"€{overage_cost_total:.2f} for {remaining_orders_after_bundles} extra orders" if remaining_orders_after_bundles != 0 else f"€{overage_cost_total:.0f} for {remaining_orders_after_bundles} extra orders"
    
    data = {
        'Description': [
            'Total Orders',
            'Small Start Cost',
            'Big Start Cost',
            'Small Prepaid Count',
            'Big Prepaid Count',
            'Small Prepaid Cost',
            'Big Prepaid Cost',
            'Overage Orders',
            'Overage Cost',
            'Total Cost'
        ],
        'Value': [
            orders,
            small_starter_description,
            big_starter_description,
            selected_small_prepaids if used_starter_bundle == 'small' else 0,
            selected_big_prepaids if used_starter_bundle == 'big' else 0,
            small_prepaid_descriptions,
            big_prepaid_descriptions,
            remaining_orders_after_bundles,
            overage_cost_str,
            f"€{total_cost:.2f}"
        ]
    }
    
    df = pd.DataFrame(data)
    return df

def plot_costs(df, title):
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.axis('off')
    ax.axis('tight')
    table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(14)
    table.scale(1.5, 1.5)

    # Pas de rijhoogte aan
    for key, cell in table.get_celld().items():
        cell.set_edgecolor('black')
        cell.set_linewidth(2)
        cell.set_facecolor('white')
        cell.set_height(0.05)  # Verhoog de rijhoogte
    
    ax.set_title(title, fontweight='bold')
    plt.show()

# Interactieve widgets
orders_slider = widgets.IntSlider(value=5000, min=10, max=10000, step=10)
start_type_dropdown = widgets.Dropdown(
    options=[('Small', 'small'), ('Big', 'big')],
    value='big',
    description=''
)
small_prepaid_count_slider = widgets.IntSlider(value=0, min=0, max=10, step=1, description='')
big_prepaid_count_slider = widgets.IntSlider(value=0, min=0, max=10, step=1, description='')

small_start_cost_slider = widgets.IntSlider(value=1000, min=500, max=2000, step=50, description='')
small_start_orders_slider = widgets.IntSlider(value=100, min=50, max=500, step=10, description='')
big_start_cost_slider = widgets.IntSlider(value=2000, min=50, max=3000, step=50, description='')
big_start_orders_slider = widgets.IntSlider(value=1350, min=50, max=2000, step=50, description='')
small_prepaid_cost_slider = widgets.IntSlider(value=250, min=100, max=500, step=50, description='')
small_prepaid_orders_slider = widgets.IntSlider(value=250, min=100, max=500, step=50, description='')
big_prepaid_cost_slider = widgets.IntSlider(value=1000, min=250, max=4000, step=100, description='')
big_prepaid_orders_slider = widgets.IntSlider(value=1100, min=250, max=1500, step=50, description='')
overage_cost_slider = widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.1, description='')

def update_custom_plot(orders, start_type, small_prepaid_count, big_prepaid_count, small_start_cost, small_start_orders, big_start_cost, big_start_orders, small_prepaid_cost, small_prepaid_orders, big_prepaid_cost, big_prepaid_orders, overage_cost):
    start_bundles = [
        (small_start_cost, small_start_orders, 'small'),
        (big_start_cost, big_start_orders, 'big')
    ]
    
    prepaid_bundles = [
        (small_prepaid_cost, small_prepaid_orders, 'small'),
        (big_prepaid_cost, big_prepaid_orders, 'big')
    ]
    
    df = display_custom_costs_df(orders, start_bundles, prepaid_bundles, overage_cost, start_type, small_prepaid_count, big_prepaid_count)
    plot_costs(df, "Pricing Model Calculator")

# Groepeer de sliders met labels erboven en voeg visuele scheiding toe
subscription_selection = widgets.VBox([
    widgets.HTML("<h2>Pricing Model Calculator</h2>"),
    widgets.HTML("<h3>Subscription and Prepaid Selection</h3>"),
    widgets.VBox([widgets.Label(value='Start Type'), start_type_dropdown]),
    widgets.VBox([widgets.Label(value='Orders'), orders_slider]),
    widgets.VBox([widgets.Label(value='Small Prepaids'), small_prepaid_count_slider]),
    widgets.VBox([widgets.Label(value='Big Prepaids'), big_prepaid_count_slider]),
])

subscription_details = widgets.VBox([
    widgets.HTML("<h3>Subscription and Prepaid Details</h3>"),
    widgets.VBox([widgets.Label(value='Small Start €'), small_start_cost_slider]),
    widgets.VBox([widgets.Label(value='Small Start Orders'), small_start_orders_slider]),
    widgets.VBox([widgets.Label(value='Big Start €'), big_start_cost_slider]),
    widgets.VBox([widgets.Label(value='Big Start Orders'), big_start_orders_slider]),
    widgets.VBox([widgets.Label(value='Small Prepaid €'), small_prepaid_cost_slider]),
    widgets.VBox([widgets.Label(value='Small Prepaid Orders'), small_prepaid_orders_slider]),
    widgets.VBox([widgets.Label(value='Big Prepaid €'), big_prepaid_cost_slider]),
    widgets.VBox([widgets.Label(value='Big Prepaid Orders'), big_prepaid_orders_slider]),
    widgets.VBox([widgets.Label(value='Overage €'), overage_cost_slider])
])

interactive_plot = widgets.interactive_output(update_custom_plot, {
    'orders': orders_slider,
    'start_type': start_type_dropdown,
    'small_prepaid_count': small_prepaid_count_slider,
    'big_prepaid_count': big_prepaid_count_slider,
    'small_start_cost': small_start_cost_slider,
    'small_start_orders': small_start_orders_slider,
    'big_start_cost': big_start_cost_slider,
    'big_start_orders': big_start_orders_slider,
    'small_prepaid_cost': small_prepaid_cost_slider,
    'small_prepaid_orders': small_prepaid_orders_slider,
    'big_prepaid_cost': big_prepaid_cost_slider,
    'big_prepaid_orders': big_prepaid_orders_slider,
    'overage_cost': overage_cost_slider
})

# CSS-stijl om de labels volledig zichtbaar te maken en sliders te centreren
display(HTML("<style>div.widget-label { min-width: 20ex !important; }</style>"))
display(widgets.VBox([subscription_selection, subscription_details, interactive_plot]))